In [1]:
'''Imports'''
%matplotlib widget

import numpy as np
from scipy.integrate import odeint
import scipy.linalg as linalg
import matplotlib.pyplot as plt

plt.rcParams.update({
    "text.usetex": True,
    "font.family": "serif",
    "font.serif": ["Times"],
    'font.size': 12
})


In [2]:
"""Mathieu's monodromy matrix"""


def mathieu(phi, t, delta, epsilon):
    # returns [\dot{u},\dot{p}] for Mathieu's equation with parameters delta, epsilon
    return [phi[1], -(delta + epsilon * np.cos(t)) * phi[0]]


def monodromy(delta, epsilon):
    # returns the monodromy matrix M for Mathieu's equation
    # with parameters delta, epsilon
    # and canonical initial conditions
    T = 2 * np.pi
    Mon = np.zeros((2, 2))

    i = 0
    for phi0 in np.eye(2):
        Mon[:, i] = odeint(mathieu, phi0, [0, T], args=(delta, epsilon))[-1, :].T
        i += 1

    return Mon


In [3]:
'''Mathieu's transition curves (execution time ~ 3 min)'''

# number of points in each direction
Nd = 1000
Ne = 100

# initialize plot range and mesh
d_min, d_max = 0, 4
e_min, e_max = 0, 4

[delta, epsilon] = np.meshgrid(np.linspace(d_min, d_max, Nd),
                               np.linspace(e_min, e_max, Ne))

# compute trace over mesh
trMon = np.zeros((Ne, Nd))

for i in range(Nd):
    for j in range(Ne):
        trMon[j, i] = np.trace(monodromy(delta[j, i], epsilon[j, i]))

# contour plot
two = 1.9999
plt.contourf(delta, epsilon, trMon, [-1000, -two, two, 1000])
plt.colorbar()
plt.contour(delta,
            epsilon,
            trMon, [-two, two],
            colors='k',
            linestyles='dashed',
            linewidths=1)
plt.xlabel(r'$\delta$')
plt.ylabel(r'$\epsilon$')
plt.yticks(range(0, 5))
plt.grid(which='major', alpha=0.3, color='w')


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [4]:
"""Butcher tableaux"""
# for two implicit symplectic Runge-Kutta methods: Radau 6 and Gauss-Legendre 6

sq6 = np.sqrt(6)
radau6 = np.array(
    [
        [0, 1 / 9, (-1 - sq6) / 18, (-1 + sq6) / 18],
        [(6 - sq6) / 10, 1 / 9, (88 + 7 * sq6) / 360, (88 - 43 * sq6) / 360],
        [(6 + sq6) / 10, 1 / 9, (88 + 43 * sq6) / 360, (88 - 7 * sq6) / 360],
        [0, 1 / 9, (16 + sq6) / 36, (16 - sq6) / 36],
    ]
)

sq15 = np.sqrt(15)
GL6 = np.array(
    [
        [1 / 2 - sq15 / 10, 5 / 36, 2 / 9 - sq15 / 15, 5 / 36 - sq15 / 30],
        [1 / 2, 5 / 36 + sq15 / 24, 2 / 9, 5 / 36 - sq15 / 24],
        [1 / 2 + sq15 / 10, 5 / 36 + sq15 / 30, 2 / 9 + sq15 / 15, 5 / 36],
        [0, 5 / 18, 4 / 9, 5 / 18],
    ]
)


In [5]:
'''Implicit Runge-Kutta Solver'''


def irk(sys, ic, tf, steps, fp, butcher, *args):
    # solves dy/dt = sys(y, t, *args), y(0) = ic on [0, tf]
    # uses Implicit Runge-Kutta with tableau = butcher 
    # ic = initial conditions, tf = final time steps = number of steps
    # fp = number of fixed-point iterations per step

    # step size
    h = tf / steps

    # initial conditions
    t = 0
    y = ic
    Y = [ic]

    # RK stages and slopes
    stages = np.shape(butcher)[0] - 1
    k = np.zeros((*np.shape(y), stages), y.dtype)

    # Implicit Runge-Kutta
    for _ in range(steps):
        # solve iteratively for slopes
        for _ in range(fp):
            for s in np.arange(stages):
                k[..., s] = sys(y + h * k @ butcher[s, 1:],
                                t + butcher[s, 0] * h, *args)
        # update rule
        y = y + h * k @ butcher[stages, 1:]
        Y += [y]
        t += h

    return Y


In [6]:
"""ODE system for a cosine-modulated Floquet-reduced chain"""


def sysQ(phi, t, Q, Z, nu, k, m):
    # returns \dot{\phi}=A(t)*\phi for a single unit cell
    # of a modulated chain with parameters Z, nu, dk, dm
    # and Floquet multiplier Q

    # initialize chain(t)
    x = 2 * np.pi / Z * np.arange(Z)
    k = (k[0] + k[1] * np.cos(x - nu * t))[:, None]
    m = (m[0] + m[1] * np.cos(x - nu * t))[:, None]

    # compute K*u
    U = phi[0:Z, :]
    T = k * (np.vstack([U[1:Z, :], Q * U[0, :]]) - U)
    KU = np.vstack([T[-1, :] / Q, T[0 : Z - 1, :]]) - T

    return np.vstack((phi[Z : 2 * Z, :] / m, -KU))


In [11]:
"""Dispersion function"""


def dispersion(Z, c, nu, k, m, substeps=20, fp=7):
    # returns the dispersion diagram (Wnb, Freq, Amp)
    # of a modulated chain (Z, c, nu, k, m)
    # substeps and fp are numerical integration parameters

    # characteristic times
    T_nat = 2 * np.pi * np.sqrt((m[0] - m[1]) / (k[0] + k[1]))
    T = 2 * np.pi / nu
    tau = T / Z

    # number of steps for numerical integration
    steps = int(tau / min(tau, T_nat)) * substeps

    # wavenumbers
    q = np.arange(0, 2 * np.pi / Z, 2 * np.pi / Z / c)

    # initialize dispersion diagram
    Wnb = np.zeros((Z * steps * c, 2 * Z))
    Freq = np.zeros((Z * steps * c, 2 * Z))
    Amp = np.zeros((Z * steps * c, 2 * Z))

    for ell in range(c):
        # integrate over [0, tau] + boundary condition: u_Z = Q u_0
        Q = np.exp(1j * q[ell] * Z)
        Phi = irk(
            sysQ, np.eye(2 * Z, dtype=np.cdouble), tau, steps, fp, GL6, Q, Z, nu, k, m
        )

        # reduced monodromy
        mon = Phi.pop()

        # shifted reduced monodromy
        newOrder = np.roll(np.arange(2 * Z).reshape(2, Z), -1, axis=1).flatten()
        stm = mon[newOrder, :]
        stm[[Z - 1, -1], :] = stm[[Z - 1, -1], :] * Q

        # eigen-values, vectors and frequencies
        ev, EV0 = np.linalg.eig(stm)
        omega = np.angle(np.exp(1j * q[ell]) / ev) / tau

        # propagate Floquet modes over [0, tau[
        EVt = Phi @ EV0

        # extend propagation of mass 0 to one full period [0, T[
        u0 = EVt[:, 0, :]
        for j in range(1, Z):
            u0 = np.concatenate((u0, EVt[:, Z - j, :] * ev ** j / Q))

        # extend time to [0, T[
        t = np.linspace(0, T, Z * steps + 1)[:-1]

        # extract periodic displacement profile
        utilde0 = u0 * np.exp(1j * omega * t[:, None])

        # compute Fourier components, normalized
        U0 = np.abs(np.fft.fft(utilde0, axis=0))
        U0 = U0 / np.max(U0, axis=0)

        # frequencies and wavenumbers
        cutoff = 2 * np.pi * Z * steps / T
        J = np.arange(Z * steps, 0, -1)[:, None]
        freq = np.mod(J * nu + omega, cutoff)
        wnb = np.mod(2 * np.pi * J / Z + q[ell], 2 * np.pi)

        # re-center on 1st Brillouin zone
        freq[freq > cutoff / 2] = freq[freq > cutoff / 2] - cutoff
        wnb[wnb > np.pi] = wnb[wnb > np.pi] - 2 * np.pi

        # store diagram
        Freq[ell * Z * steps : (ell + 1) * Z * steps, :] = freq
        Wnb[ell * Z * steps : (ell + 1) * Z * steps, :] = wnb
        Amp[ell * Z * steps : (ell + 1) * Z * steps, :] = U0

    return Wnb, Freq, Amp


In [12]:
"""Dispersion function for non-modulated chain"""


def dispersionNM(Z, c, k, m):
    # returns the dispersion diagram (Wnb, Freq, Amp)
    # of a chain (Z, c, k, m)

    # wavenumbers
    q = np.arange(-np.pi / Z, np.pi / Z, 2 * np.pi / Z / c)

    # initialize dispersion diagram
    Wnb = np.zeros((Z * c, 2 * Z))
    Freq = np.zeros((Z * c, 2 * Z))
    Amp = np.zeros((Z * c, 2 * Z))

    for ell in range(c):
        # integrate over [0, tau] + boundary condition: u_Z = Q u_0
        Q = np.exp(1j * q[ell] * Z)
        Sys = sysQ(np.eye(2 * Z, dtype=np.cdouble), 0, Q, Z, 0, k, m)

        # eigen-values, vectors and frequencies
        ev, EV0 = np.linalg.eig(Sys)
        omega = -np.imag(ev)

        # compute Fourier components, normalized
        U0 = np.abs(
            np.fft.fft(
                EV0[:Z, :] * np.exp(-1j * np.arange(Z)[:, None] * q[ell]), axis=0
            )
        )
        U0 = U0 / np.max(U0, axis=0)

        # store diagram
        Freq[ell * Z : (ell + 1) * Z, :] = omega
        wnb = np.mod(2 * np.pi * np.arange(Z) / Z + q[ell], 2 * np.pi)
        wnb[wnb > np.pi] = wnb[wnb > np.pi] - 2 * np.pi
        Wnb[ell * Z : (ell + 1) * Z, :] = wnb[:, None]
        Amp[ell * Z : (ell + 1) * Z, :] = U0

    return Wnb, Freq, Amp


In [13]:
"""Dispersion diagrams"""

tol = 1e-3

fig1, (ax1, ax2, ax3, ax4) = plt.subplots(1,4, sharex=True, sharey=True)

Z, nu, k, m = 4, 0.4, [1, 0.0], [1, 0.0]
c = 30
Wnb, Freq, Amp = dispersion(Z, c, nu, k, m,substeps=40)

aspect = 1.3

ax1.scatter(Wnb[Amp>tol], Freq[Amp>tol], Amp[Amp>tol])
ax1.set_xlabel(r'$q$')
ax1.set_ylabel(r'$\omega$')
ax1.grid(which='major', alpha=0.3, color='gray')
ax1.set_aspect(aspect)

Z, k, m = 4, [1, 0.4], [1, 0.4]
c = 30
Wnb, Freq, Amp = dispersionNM(Z, c, k, m)

ax2.scatter(Wnb[Amp>tol], Freq[Amp>tol], Amp[Amp>tol])
ax2.set_xlabel(r'$q$')
ax2.grid(which='major', alpha=0.3, color='gray')
ax2.set_aspect(aspect)

Z, nu, k, m = 4, 0.4, [1, 0.4], [1, 0.4]
c = 30
Wnb, Freq, Amp = dispersion(Z, c, nu, k, m,substeps=40)

ax3.scatter(Wnb[Amp>tol], Freq[Amp>tol], Amp[Amp>tol])
ax3.set_xlabel(r'$q$')
ax3.grid(which='major', alpha=0.3, color='gray')
ax3.set_xticks([-np.pi,0,np.pi])
ax3.set_xticklabels([r'$-\pi$',r'$0$',r'$\pi$'])
ax3.set_aspect(aspect)

Z, nu, k, m = 4, 4.564, [1, 0.4], [1, 0.4]
c = 30
Wnb, Freq, Amp = dispersion(Z, c, nu, k, m,substeps=40)

tol = 1e-1
ax4.scatter(Wnb[Amp>tol], Freq[Amp>tol], Amp[Amp>tol])
ax4.set_xlabel(r'$q$')
ax4.grid(which='major', alpha=0.3, color='gray')
ax4.set_aspect(aspect)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [14]:
"""ODE system for a cosine-modulated chain"""


def sys(phi, t, N, Z, nu, k, m):
    # returns \dot{\phi}=A(t)*\phi for a modulated chain
    # with parameters N, Z, nu, k, m
    # k, m = [average, amplitude]

    # initialize chain(t)
    x = 2 * np.pi / Z * np.arange(N)
    k = (k[0] + k[1] * np.cos(x - nu * t))[:, None]
    m = (m[0] + m[1] * np.cos(x - nu * t))[:, None]

    # compute K*u
    U = phi[0:N, :]
    T = k * (np.vstack([U[1:N, :], U[0, :]]) - U)
    KU = np.vstack([T[-1, :], T[0 : N - 1, :]]) - T

    return np.vstack((phi[N : 2 * N, :] / m, -KU))


In [15]:
"""Monodromy"""


def monodromy(Z, nu, k, m, substeps=40, fp=7, c=200, fakenu=1):

    # time scales
    T_nat = 2 * np.pi * np.sqrt(m[0] / k[0])
    if nu == 0:
        T = 2 * np.pi / fakenu
        tau = T
    else:
        T = 2 * np.pi / nu
        tau = T/Z

    # number of steps
    steps = int(tau / min(tau, T_nat)) * substeps

    # number of nodes
    N = c * Z

    # integrate over [0, tau]
    Phi = irk(sys, np.eye(2 * N), tau, steps, fp, GL6, N, Z, nu, k, m)

    # reduced monodromy
    mon = Phi.pop()

    if nu == 0:
        return mon
    else:
        # shifted reduced monodromy
        newOrder = np.roll(np.arange(2 * N).reshape(2, N), -1, axis=1).flatten()
        stm = mon[newOrder, :]

        # monodromy
        Mon = np.linalg.matrix_power(stm, Z)
        newOrder = np.roll(np.arange(2 * N).reshape(2, N), Z, axis=1).flatten()
        return Mon[newOrder, :]


In [16]:
"""Propagate initial conditions"""


def propagate(Mon, phi0, M):
    N = np.size(phi0) // 2

    U = [phi0[:N]]
    P = [phi0[N:]]

    for _ in range(M - 1):
        phi0 = Mon @ phi0
        U += [phi0[:N]]
        P += [phi0[N:]]

    U = np.array(U)
    P = np.array(P)
    U = U / np.max(np.abs(U))
    P = P / np.max(np.abs(P))

    return U, P


In [17]:
"""Plots of propagated signals (execution time ~ 5 min)"""

Z = 4
c = 200
N = c * Z

fig2, ax = plt.subplots(2,4, sharex=True, sharey='row')
ax[0, 0].set_xticks([0, 200, 400, 600, 800])

mon1 = monodromy(Z, 0., [1, 0], [1, 0], c = c, fakenu = 0.4)
mon2 = monodromy(Z, 0., [1, 0.4], [1, 0.4], c = c, fakenu = 0.4)
mon3 = monodromy(Z, 0.4, [1, 0.4], [1, 0.4], c = c)
mon4 = monodromy(Z, 4.564, [1, 0.4], [1, 0.4], c = c)

## first initial condition
phi0 = np.random.randn(2 * N)

# 1st plot
M = 250
Mfast = int(M/(0.4/4.564))

U, P = propagate(mon1, phi0, M)

x = np.arange(N)
t = np.arange(M)
x, t = np.meshgrid(x, t)

ax[0, 0].scatter(x.ravel(), t.ravel(),10*np.abs(P).ravel(),color=np.hstack((np.ones((N*M,3))*np.array([0.5,0.1,0.1])[None,:], 0.2*np.abs(P).ravel()[:,None])), marker=".")
ax[0, 0].set_ylabel(r'time [$T$]')

# 2nd plot
U, P = propagate(mon2, phi0, M)

x = np.arange(N)
t = np.arange(M)
x, t = np.meshgrid(x, t)

ax[0, 1].scatter(x.ravel(), t.ravel(),10*np.abs(P).ravel(),color=np.hstack((np.ones((N*M,3))*np.array([0.5,0.1,0.1])[None,:], 0.2*np.abs(P).ravel()[:,None])), marker=".")

# 3rd plot
U, P = propagate(mon3, phi0, M)

x = np.arange(N)
t = np.arange(M)
x, t = np.meshgrid(x, t)

ax[0, 2].scatter(x.ravel(), t.ravel(),10*np.abs(P).ravel(),color=np.hstack((np.ones((N*M,3))*np.array([0.5,0.1,0.1])[None,:], 0.2*np.abs(P).ravel()[:,None])), marker=".")

# 4th plot
U, P = propagate(mon4, phi0, Mfast)

x = np.arange(N)
t = np.arange(Mfast)
x, t = np.meshgrid(x, t)

ax[0, 3].scatter(x.ravel(), M*t.ravel()/Mfast,0.2*np.abs(P).ravel(),color=np.hstack((np.ones((N*Mfast,3))*np.array([0.5,0.1,0.1])[None,:], 0.2*np.abs(P).ravel()[:,None])), marker=".")

## second initial condition
phi0 = np.zeros(2 * N)
phi0[N//2] = 1

# 5th plot
U, P = propagate(mon1, phi0, M)

x = np.arange(N)
t = np.arange(M)
x, t = np.meshgrid(x, t)

ax[1, 0].scatter(x.ravel(), t.ravel(),50*np.abs(P).ravel(),color=np.hstack((np.ones((N*M,3))*np.array([0.5,0.1,0.1])[None,:], 0.2*np.abs(P).ravel()[:,None])), marker=".")
ax[1, 0].set_ylabel(r'time [$T$]')
ax[1, 0].set_xlabel(r'mass index')

# 6th plot
U, P = propagate(mon2, phi0, M)

x = np.arange(N)
t = np.arange(M)
x, t = np.meshgrid(x, t)

ax[1, 1].scatter(x.ravel(), t.ravel(),50*np.abs(P).ravel(),color=np.hstack((np.ones((N*M,3))*np.array([0.5,0.1,0.1])[None,:], 0.2*np.abs(P).ravel()[:,None])), marker=".")
ax[1, 1].set_xlabel(r'mass index')

# 7th plot
U, P = propagate(mon3, phi0, M)

x = np.arange(N)
t = np.arange(M)
x, t = np.meshgrid(x, t)

ax[1, 2].scatter(x.ravel(), t.ravel(),50*np.abs(P).ravel(),color=np.hstack((np.ones((N*M,3))*np.array([0.5,0.1,0.1])[None,:], 0.2*np.abs(P).ravel()[:,None])), marker=".")
ax[1, 2].set_xlabel(r'mass index')

# 8th plot
U, P = propagate(mon4, phi0, Mfast)

x = np.arange(N)
t = np.arange(Mfast)
x, t = np.meshgrid(x, t)

ax[1, 3].scatter(x.ravel(), M*t.ravel()/Mfast,np.abs(P).ravel(),color=np.hstack((np.ones((N*Mfast,3))*np.array([0.5,0.1,0.1])[None,:], 0.2*np.abs(P).ravel()[:,None])), marker=".")
ax[1, 3].set_xlabel(r'mass index')


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 0, 'mass index')

In [19]:
"""Dispersion diagrams - continuous case (execution time ~ 1 min)"""

tol = 1e-4

fig1, ax = plt.subplots(1, 1, sharex='row', sharey='row')

aspect = 2.6

i = 0

Z, k, m = 100, np.array([1, 0.95]), np.array([1, 0.95])
c = 100
s = 1

ax.set_ylabel(r"$\omega$")
ax.set_xlim([-2 * np.pi, 2 * np.pi])
ax.set_ylim([-6.5, 6.5])

for smod in [0.6]: #, 1.0001, 1.4, 4]:
    nu = 2 * np.pi * smod / Z
    Wnb, Freq, Amp = dispersion(Z, c, nu, k, m)

    ax.scatter(
        Z * Wnb[Amp > tol],
        Z * Freq[Amp > tol],
        Amp[Amp > tol]
    )
    ax.set_xlabel(r"$qL$")
    ax.grid(which="major", alpha=0.3, color="gray")
    ax.set_aspect(aspect)

    i += 1


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [21]:
"""Plots of propagated signals - Willis effect (execution time ~ 40 min)"""

fig2, ax = plt.subplots(1, 1, sharex='row', sharey='row')

aspect = 2.6

i = 0

Z, k, m = 40, np.array([1, 0.95]), np.array([1, 0.95])
c = 100
s = 1

N = c*Z

mon = []

M = 50

x = np.arange(N)

phi0 = np.zeros(2 * N)
phi0[:N] = 500*((x-N//2)/10/Z)*np.exp(-((x-N//2)/10/Z)**2)
phi0[N:] = np.exp(-((x-N//2)/10/Z)**2)


t = np.arange(M)
x, t = np.meshgrid(x, t)


for smod in [0.6]: #, 1.0001, 1.4, 4]:
    nu = 2 * np.pi * smod / Z
    mon = mon + [monodromy(Z,nu,k,m,40,7,c)]
    U, P = propagate(mon[i], phi0, M)

    ax.scatter(x.ravel(), t.ravel(),50*np.abs(P).ravel()**3,color=np.hstack((np.ones((N*M,3))*np.array([0.5,0.1,0.1])[None,:], 0.2*np.abs(P).ravel()[:,None])), marker=".")
    ax.set_ylabel(r'time [$T$]')
    ax.set_xlabel('mass index')


    i += 1

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [22]:
"""Dispersion diagrams - continuous case - fictitious medium"""

tol = 1e-4

fig3, ax = plt.subplots(1, 2, sharex='row', sharey='row')

aspect = 2.6

i = 0

Z, k, m = 20, np.array([1, 0.95]), np.array([1, 0.95])
c = 100
s = 1

ax[i].set_ylabel(r"$\omega$")
ax[i].set_xlim([-2 * np.pi, 2 * np.pi])
ax[i].set_ylim([-6.5, 6.5])

for smod in [0, 0.6]: #, 1.0001, 1.4, 4]:
    nu = 2 * np.pi * smod / Z
    s = 1
    Wnb, Freq, Amp = dispersionNM(Z, c, m*(s**2-smod**2), k/(s**2-smod**2))

    ax[i].scatter(
        Z * Wnb[Amp > tol],
        Z * Freq[Amp > tol],
        Amp[Amp > tol]
    )
    ax[i].set_xlabel(r"$qL$")
    ax[i].grid(which="major", alpha=0.3, color="gray")
    ax[i].set_aspect(aspect)

    # ax[1,i].scatter(np.real(Mult), np.imag(Mult), c=1/abs(Mult - 1))
    # ax[1,i].set_aspect(1)
    i += 1


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …